1. Mount Drive and clone the dir

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/"

/content/drive/MyDrive


In [ ]:
!git clone https://github.com/HoyoenKim/CMI_Detection.git

Cloning into 'CMI_Detection'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 34 (delta 1), reused 34 (delta 1), pack-reused 0
Receiving objects: 100% (34/34), 17.43 KiB | 1.34 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
cd "/content/drive/MyDrive/CMI_Detection"

/content/drive/MyDrive/CMI_Detection


In [ ]:
!pip list | grep polar

polars                           0.17.3


In [ ]:
!pip install polars==0.19.15

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 44.6 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.19.0
    Uninstalling polars-0.19.0:
      Successfully uninstalled polars-0.19.0


In [9]:
import polars as pl

# Load the data
series_df = pl.read_parquet("./raw_data/train_series.parquet")
series_df.sort(by=["series_id", "timestamp"])

# Calculate the differences
series_df = series_df.with_columns([
    pl.col('anglez'),
    pl.col('enmo'),
    ((pl.col('anglez') - pl.col('anglez').shift(1)).fill_nan(0)).alias('anglez_diff'),
    ((pl.col('enmo') - pl.col('enmo').shift(1)).fill_nan(0)).alias('enmo_diff')
])

# Calculate mean and standard deviation for the differences
angle_stats = series_df.select([
    pl.mean('anglez').alias('anglez_mean'),
    pl.std('anglez').alias('anglez_std')
]).to_dict()

enmo_stats = series_df.select([
    pl.mean('enmo').alias('enmo_mean'),
    pl.std('enmo').alias('enmo_std')
]).to_dict()

angle_diff_stats = series_df.select([
    pl.mean('anglez_diff').alias('anglez_diff_mean'),
    pl.std('anglez_diff').alias('anglez_diff_std')
]).to_dict()

enmo_diff_stats = series_df.select([
    pl.mean('enmo_diff').alias('enmo_diff_mean'),
    pl.std('enmo_diff').alias('enmo_diff_std')
]).to_dict()

# Extract the values
ANGLE_MEAN = angle_stats['anglez_mean'][0]
ANGLE_STD  = angle_stats['anglez_std'][0]
ENMO_MEAN  =  enmo_stats['enmo_mean'][0]
ENMO_STD   =  enmo_stats['enmo_std'][0]

ANGLE_DIFF_MEAN = angle_diff_stats['anglez_diff_mean'][0]
ANGLE_DIFF_STD = angle_diff_stats['anglez_diff_std'][0]
ENMO_DIFF_MEAN = enmo_diff_stats['enmo_diff_mean'][0]
ENMO_DIFF_STD = enmo_diff_stats['enmo_diff_std'][0]

# Print the calculated values
print("ANGLEZ_MEAN:", ANGLE_MEAN)
print("ANGLEZ_STD:",  ANGLE_STD)
print("  ENMO_MEAN:",  ENMO_MEAN)
print("  ENMO_STD:",   ENMO_STD)

print("ANGLEZ_DIFF_MEAN:", ANGLE_DIFF_MEAN)
print("ANGLEZ_DIFF_STD:", ANGLE_DIFF_STD)
print("ENMO_DIFF_MEAN:", ENMO_DIFF_MEAN)
print("ENMO_DIFF_STD:", ENMO_DIFF_STD)

ANGLEZ_MEAN: -8.810476303100586
ANGLEZ_STD: 35.52186965942383
  ENMO_MEAN: 0.041315071284770966
  ENMO_STD: 0.10182943940162659
ANGLEZ_DIFF_MEAN: -2.669245020570088e-07
ANGLEZ_DIFF_STD: 11.40933895111084
ENMO_DIFF_MEAN: -9.2523896530472e-12
ENMO_DIFF_STD: 0.08300307393074036


2. Preprocess the data

In [ ]:
from preprocess.preprocess import preprocess

train_series_path = "./raw_data/train_series.parquet"
train_series_save_dir = "./data"
do_preprocess = True
if do_preprocess:
    preprocess(train_series_path, train_series_save_dir)

100%|██████████| 277/277 [01:42<00:00,  2.71it/s]


3. Train the model

In [ ]:
!pip install pytorch_lightning segmentation_models_pytorch==0.3.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 63.0 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=a3fe259f0d46c01870e2e5d49b70c45adee53ba47cbe6b53c4f29ffdcb185a22
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=f24fad797f9f8f3de410a46065b0cc007ccb1fe2d443bdc3bd687c195861022f
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5

In [ ]:
#https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/discussion/456177
train_series_ids = [
"038441c925bb",
"03d92c9f6f8a",
"0402a003dae9",
"04f547b8017d",
"05e1944c3818",
"062cae666e2a",
"062dbd4c95e6",
"08db4255286f",
"0a96f4993bd7",
"0cd1e3d0ed95",
"0ce74d6d2106",
"0cfc06c129cc",
"0d0ad1e77851",
"0dee4fda51c3",
"0ec9fc461819",
"0ef7d94fde99",
"0f572d690310",
"0f9e60a8e56d",
"10469f6765bf",
"1087d7b0ff2e",
"10f8bc1f7b07",
"12d01911d509",
"1319a1935f48",
"137771d19ca2",
"137b99e936ab",
"13b4d6a01d27",
"148471991ffb",
"154fe824ed87",
"16fe2798ed0f",
"1716cd4163b2",
"1762ab70ec76",
"188d4b7cd28b",
"18a0ca03431d",
"18b61dd5aae8",
"1955d568d987",
"1b92be89db4c",
"1c7c0bad1263",
"1d4569cbac0f",
"1e6717d93c1d",
"1f96b9668bdf",
"207eded97727",
"25e2b3dd9c3b",
"2654a87be968",
"27f09a6a858f",
"280e08693c6d",
"292a75c0b94e",
"29c75c018220",
"29d3469bd15d",
"2b0a1fa8eba8",
"2b8d87addea9",
"2cd2340ca14d",
"2e9ced2c7976",
"2f7504d0f426",
"2fbbee1a38e3",
"2fc653ca75c7",
"31011ade7c0a",
"3318a0e3ed6f",
"33ceeba8918a",
"3452b878e596",
"349c5562ee2c",
"35826366dfc7",
"361366da569e",
"3664fe9233f9",
"3665c86afaf5",
"390b487231ce",
"3a9a9dc2cbd9",
"3aceb17ef7bd",
"3be1545083b7",
"3be2f86c3e45",
"3c336d6ba566",
"3d53bfea61d6",
"3df0da2e5966",
"405df1b41f9f",
"40dce6018935",
"416354edd92a",
"449766346eb1",
"44a41bba1ee7",
"44d8c02b369e",
"4743bdde25df",
"483d6545417f",
"4a31811f3558",
"4ab54be1a403",
"4ac356361be9",
"4b45c36f8f5a",
"4feda0596965",
"519ae2d858b0",
"51b23d177971",
"51c49c540b4e",
"51fdcc8d9fe7",
"559ffb7c166a",
"55a47ff9dc8a",
"55b7f5c99930",
"599ca4ed791b",
"5aad18e7ce64",
"5acc9d63b5fd",
"5c088d7e916c",
"5c55a5e717d6",
"5e816f11f5c3",
"5f40907ec171",
"5f76965e10cf",
"5f94bb3e1bed",
"5ffd5e1e81ac",
"601559e1777d",
"60d31b0bec3b",
"60e51cad2ffb",
"612aa8ba44e2",
"653622ac8363",
"655f19eabf1e",
"67f5fc60e494",
"694faf956ebf",
"6a4cd123bd69",
"6bf95a3cf91c",
"6ca4f4fca6a2",
"6d6b9d22d48a",
"6ee4ade1f2bd",
"702bb5387b1e",
"703b5efa9bc1",
"72ba4a8afff4",
"72bbd1ac3edf",
"72d2234e84e4",
"73fb772e50fb",
"7476c0bd18d2",
"7504165f497d",
"752900afe3a6",
"76237b9406d5",
"77ca4db83644",
"7822ee8fe3ec",
"78569a801a38",
"785c9ca4eff7",
"7df249527c63",
"7fd4284b7ee8",
"804594bb1f06",
"808652a666c6",
"83fa182bec3a",
"844f54dcab89",
"854206f602d0",
"87a6cbb7c4ed",
"8877a6586606",
"8898e6db816d",
"89bd631d1769",
"89c7daa72eee",
"8a22387617c3",
"8a306e0890c0",
"8b159a98f485",
"8b8b9e29171c",
"8becc76ea607",
"8e32047cbc1f",
"8f6f15b9f598",
"8fb18e36697d",
"90eac42a9ec9",
"91127c2b0e60",
"91cb6c98201f",
"9277be28a1cf",
"927dd0c35dfd",
"939932f1822d",
"971207c6a525",
"99237ce045e4",
"99b829cbad2d",
"9a340507e36a",
"9aed9ee12ae2",
"9b9cd7b7af8c",
"9c91c546e095",
"9ddd40f2cb36",
"9ee455e4770d",
"9fbdeffbe2ba",
"a167532acca2",
"a261bc4b7470",
"a2b0a64ec9cf",
"a3e59c2ce3f6",
"a4e48102f402",
"a596ad0b82aa",
"a681f9b04b21",
"a81f4472c637",
"a88088855de5",
"a9a2f7fac455",
"a9e5f5314bcb",
"aa81faa78747",
"ad425f3ee76d",
"aed3850f65f0",
"af91d9a50547",
"b1831c4979da",
"b364205aba43",
"b4b75225b224",
"b7188813d58a",
"b737f8c78ec5",
"b750c8c1556c",
"b7fc34995d0f",
"b84960841a75",
"ba8083a2c3b8",
"bb5612895813",
"bccf2f2819f8",
"bdfce9ce62b9",
"bf00506437aa",
"bfa54bd26187",
"bfe41e96d12f",
"c107b5789660",
"c289c8a823e0",
"c3072a759efb",
"c38707ef76df",
"c535634d7dcd",
"c5365a55ebb7",
"c5d08fc3e040",
"c6788e579967",
"c68260cc9e8f",
"c75b4b207bea",
"c7b1283bb7eb",
"c7b2155a4a47",
"c7d693f24684",
"c8053490cec2",
"c908a0ad3e31",
"ca730dbf521d",
"ca732a3c37f7",
"cca14d1966c1",
"ccdee561ee5d",
"ce85771a714c",
"ce9164297046",
"cf13ed7e457a",
"cfeb11428dd7",
"d043c0ca71cd",
"d0f613c700f7",
"d150801f3145",
"d25e479ecbb7",
"d2d6b9af0553",
"d2fef7e4defd",
"d3dddd3c0e00",
"d515236bdeec",
"d5be621fd9aa",
"d5e47b94477e",
"d8de352c2657",
"d93b0c7de16b",
"d9e887091a5c",
"dacc6d652e35",
"db5e0ee1c0ab",
"db75092f0530",
"dc80ca623d71",
"de6fedfb6139",
"def21f50dd3c",
"df33ae359fb5",
"dfc3ccebfdc9",
"dff367373725",
"e0686434d029",
"e0d7b0dcf9f3",
"e11b9d69f856",
"e1f2a4f991cb",
"e1f5abb82285",
"e2a849d283c0",
"e2b60820c325",
"e30cb792a2bc",
"e34b496b84ce",
"e4500e7e19e1",
"e586cbfa7762",
"e69aff66e0cb",
"e6ddbaaf0639",
"e867b5133665",
"e8d0a37c3eba",
"ea0770830757",
"ebb6fae8ed43",
"ebd76e93ec7d",
"ece2561f07e9",
"ee4e0e3afd3d",
"eec197a4bdca",
"eef041dd50aa",
"efbfc4526d58",
"f0482490923c",
"f2c2436cf7b7",
"f564985ab692",
"f56824b503a0",
"f6d2cc003183",
"f7eb179216c2",
"f88e18cb4100",
"f8a8da8bdd00",
"f981a0805fd0",
"fa149c3c4bde",
"fb223ed2278c",
"fbf33b1a2c10",
"fcca183903b7",
"fe90110788d2",
]

valid_series_ids =[
  "03d92c9f6f8a",
  "0cd1e3d0ed95",
  "0cfc06c129cc",
  "12d01911d509",
  "1319a1935f48",
  "1762ab70ec76",
  "280e08693c6d",
  "29c75c018220",
  "2cd2340ca14d",
  "2fbbee1a38e3",
  "33ceeba8918a",
  "35826366dfc7",
  "3665c86afaf5",
  "390b487231ce",
  "4ac356361be9",
  "4feda0596965",
  "55b7f5c99930",
  "599ca4ed791b",
  "5e816f11f5c3",
  "5f94bb3e1bed",
  "601559e1777d",
  "653622ac8363",
  "6a4cd123bd69",
  "76237b9406d5",
  "8898e6db816d",
  "89bd631d1769",
  "89c7daa72eee",
  "8becc76ea607",
  "8f6f15b9f598",
  "8fb18e36697d",
  "99b829cbad2d",
  "9b9cd7b7af8c",
  "9ddd40f2cb36",
  "9fbdeffbe2ba",
  "a88088855de5",
  "b1831c4979da",
  "b7fc34995d0f",
  "bb5612895813",
  "bccf2f2819f8",
  "bdfce9ce62b9",
  "c908a0ad3e31",
  "ccdee561ee5d",
  "d8de352c2657",
  "def21f50dd3c",
  "e2a849d283c0",
  "e2b60820c325",
  "ebb6fae8ed43",
  "ee4e0e3afd3d",
  "eec197a4bdca",
  "eef041dd50aa",
  "f0482490923c",
  "f564985ab692",
  "f56824b503a0",
  "f7eb179216c2",
  "fbf33b1a2c10",
]

In [ ]:
from train.train import train

# train
train_dir_config = {
    "processed_dir": './data',
    "train_events_path": './raw_data/train_events.csv'
}
split_config = {
    "train_series_ids": train_series_ids,
    "valid_series_ids": valid_series_ids
}
dataset_config = {
    "name": "seg",
    "batch_size": 32,
    "num_workers": 12,
    "offset": 10,
    "sigma": 10,
    "bg_sampling_rate": 0.5
}
aug_config = {
    "mixup_prob": 0.0,
    "mixup_alpha": 0.4,
    "cutmix_prob": 0.0,
    "cutmix_alpha": 0.4,
}
postprocess_config = {
    "score_th": 0.005,
    "distance": 40,
}
optimizer_config = {
    "lr": 0.0005,
}
scheduler_config = {
    "num_warmup_steps": 0,
}
model_config = {
    "name": "Spec2DCNN",
    "params": {
        "encoder_name": "resnet34",
        "encoder_weights": "imagenet",
    }
}
feature_extractor_config = {
    "name": "CNNSpectrogram",
    "params": {
        "base_filters": 64,
        "kernel_sizes": [
            32,
            16,
            2, #down_sample rate
        ],
        "stride": 2, #down_sample rate
        "sigmoid": True,
        "reinit": True,
    }
}
decoder_config = {
    "name": "UNet1DDecoder",
    "params": {
        "bilinear": False,
        "se": False,
        "res": False,
        "scale_factor": 2,
        "dropout": 0.2,
    }
}
trainer_config = {
    "epochs": 100,
    "accelerator": "auto",
    "use_amp": True,
    "debug": False,
    "gradient_clip_val": 1.0,
    "accumulate_grad_batches": 1,
    "monitor": "val_loss",
    "monitor_mode": "min",
    "check_val_every_n_epoch": 1,
}
features = [
    "anglez",
    "enmo",
    #"step",
    #"month_sin",
    #"month_cos",
    "hour_sin",
    "hour_cos",
    #"minute_sin",
    #"minute_cos",
    #"anglez_sin",
    #"anglez_cos",
]
labels = [
    "awake",
    "event_onset",
    "event_wakeup",
]
train_config = {
    "seed": 42,
    "dir": train_dir_config,
    "split": split_config,
    "features": features,
    "labels": labels,
    "duration": 5760,
    "downsample_rate": 2,
    "upsample_rate": 1,
    "dataset": dataset_config,
    "aug": aug_config,
    "pp": postprocess_config,
    "optimizer": optimizer_config,
    "scheduler": scheduler_config,
    "model": model_config,
    "feature_extractor": feature_extractor_config,
    "decoder": decoder_config,
    "exp_name": "exp00",
    "trainer": trainer_config,
}
do_train = True
if do_train:
    train(train_config)

INFO:lightning_fabric.utilities.seed:Seed set to 42
Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 65.4MB/s]
/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPU

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                    ┃ Type              ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model                   │ Spec2DCNN         │ 43.6 M │
│ 1 │ model.feature_extractor │ CNNSpectrogram    │ 13.0 K │
│ 2 │ model.encoder           │ UnetPlusPlus      │ 26.1 M │
│ 3 │ model.decoder           │ UNet1DDecoder     │ 17.5 M │
│ 4 │ model.loss_fn           │ BCEWithLogitsLoss │      0 │
└───┴─────────────────────────┴───────────────────┴────────┘

Trainable params: 43.6 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 43.6 M                                                                                               
Total estimated model params size (MB): 174

Output()

Saved best model inf -> 0.04286850046215428

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 86, global step 12963: 'val_loss' reached 0.01709 (best 0.01709), saving model to '/content/drive/MyDrive/CMI_Detection/lightning_logs/version_8/checkpoints/epoch=86-step=12963.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 87, global step 13112: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 88, global step 13261: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 89, global step 13410: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 90, global step 13559: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 91, global step 13708: 'val_loss' was not in top 1


Saved best model 0.01703939150261372 -> 0.0169207247637206

INFO:pytorch_lightning.utilities.rank_zero:Epoch 92, global step 13857: 'val_loss' reached 0.01697 (best 0.01697), saving model to '/content/drive/MyDrive/CMI_Detection/lightning_logs/version_8/checkpoints/epoch=92-step=13857.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 93, global step 14006: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 94, global step 14155: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 95, global step 14304: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 96, global step 14453: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 97, global step 14602: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 98, global step 14751: 'val_loss' was not in top 1


Saved best model 0.0169207247637206 -> 0.016888507274514367

INFO:pytorch_lightning.utilities.rank_zero:Epoch 99, global step 14900: 'val_loss' reached 0.01693 (best 0.01693), saving model to '/content/drive/MyDrive/CMI_Detection/lightning_logs/version_8/checkpoints/epoch=99-step=14900.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=14900` reached.
